Việc cài đặt môi trường `stanford_openie` tại [link](https://github.com/philipperemy/stanford-openie-python?fbclid=IwY2xjawFT3GZleHRuA2FlbQIxMAABHV6dU1rsogSgiZ7-jGH7-L639I9KWMAl-3LnnrcvA1uHueczNje8MP3x8A_aem_p_bNrr-99987NPf-Hb8uYg) không được

Sử dụng thư viện `stanfordnlp` hoặc các thư viện liên quan để thực hiện Stanford OIE.

---


Hiện tại, thư viện `stanfordnlp` đã được thay thế bằng `stanza`, nên mình sẽ sử



dụng thư viện này để trích xuất thông tin từ Wikipedia.

In [1]:
# Cài đặt stanza và wikipedia-api
!pip install stanza wikipedia-api

# Tải dữ liệu tiếng Việt cho stanza (nếu cần, có thể bỏ qua nếu sử dụng tiếng Anh)
import stanza
stanza.download('vi')


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 14.0 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14347 sha256=9685ec43f57d8c09a311b029958cc6e7b7ac91e44c683979e80c901c1bc90a61
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: vi (Vietnamese) ...


INFO:stanza:Downloaded file to /root/stanza_resources/vi/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [2]:
!pip install networkx


In [3]:
import wikipediaapi
import stanza
import matplotlib.pyplot as plt
import networkx as nx
import re

# Cung cấp thông tin user agent
user_agent = "MyApp/1.0 (https://mywebsite.com; contact@example.com)"

# Khởi tạo đối tượng Wikipedia với ngôn ngữ 'vi' và user agent
wiki_wiki = wikipediaapi.Wikipedia(language='vi', user_agent=user_agent)

# Truy vấn một trang Wikipedia
page = wiki_wiki.page("Thể thao điện tử")

# Kiểm tra xem trang có tồn tại hay không
if page.exists():
    # In ra tiêu đề trang
    print(f"Title: {page.title}")

    # In ra phần tóm tắt nội dung
    print(f"Summary: {page.summary[0:500]}")  # Lấy 500 ký tự đầu tiên của phần tóm tắt

    # In ra toàn bộ nội dung trang
    print(page.text)
else:
    print("Trang không tồn tại")


Title: Thể thao điện tử
Summary: Thể thao điện tử (Tiếng Anh: esports) là hình thức tổ chức trận đấu điện tử giữa nhiều người chơi, đặc biệt giữa những tuyển thủ chuyên nghiệp. Các thể loại trò chơi video phổ biến nhất liên quan đến thể thao điện tử là cuộc chiến đấu  trực tuyến với sự tham gia của nhiều người chơi, trò chơi chiến đấu sử dụng chiến lược thời gian thực (MOBA), và thể loại trò chơi điện tử bắn súng góc nhìn người thứ nhất (FPS), nhưng thể loại FPS chỉ được gọi là đặc trưng của E-Sport khi chưa phát triển, khi các
Thể thao điện tử (Tiếng Anh: esports) là hình thức tổ chức trận đấu điện tử giữa nhiều người chơi, đặc biệt giữa những tuyển thủ chuyên nghiệp. Các thể loại trò chơi video phổ biến nhất liên quan đến thể thao điện tử là cuộc chiến đấu  trực tuyến với sự tham gia của nhiều người chơi, trò chơi chiến đấu sử dụng chiến lược thời gian thực (MOBA), và thể loại trò chơi điện tử bắn súng góc nhìn người thứ nhất (FPS), nhưng thể loại FPS chỉ được gọi là đặc trưng của E-

In [4]:
# Khởi tạo công cụ phân tích cú pháp Stanza cho tiếng Việt
nlp = stanza.Pipeline('vi')

page_text = page.text[:1000]  # Lấy một đoạn văn bản (giới hạn 1000 ký tự để thử nghiệm)
# Phân tích văn bản bằng Stanza
doc = nlp(page_text)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: vi (Vietnamese):
| Processor    | Package       |
--------------------------------
| tokenize     | vtb           |
| pos          | vtb_charlm    |
| lemma        | identity      |
| constituency | vlsp22_charlm |
| depparse     | vtb_charlm    |
| sentiment    | vsfc_charlm   |
| ner          | vlsp          |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tru

In [5]:
# Hàm làm sạch văn bản
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ các ký tự đặc biệt, dấu câu
    return text.strip()

In [6]:
# Trích xuất và in ra kết quả các bộ ba từ câu
for sentence in doc.sentences:
    for word in sentence.words:
        cleaned_word = clean_text(word.text)
        cleaned_lemma = clean_text(word.lemma)
        print(f'word: {cleaned_word}, lemma: {cleaned_lemma}, pos: {word.pos}')

word: Thể, lemma: Thể, pos: NOUN
word: thao, lemma: thao, pos: NOUN
word: điện tử, lemma: điện tử, pos: NOUN
word: Tiếng An, lemma: Tiếng An, pos: PROPN
word: h, lemma: h, pos: PUNCT
word: esport, lemma: esport, pos: PROPN
word: s, lemma: s, pos: PUNCT
word: là, lemma: là, pos: AUX
word: hình thức, lemma: hình thức, pos: NOUN
word: tổ chức, lemma: tổ chức, pos: VERB
word: trận, lemma: trận, pos: NOUN
word: đấu điện tử, lemma: đấu điện tử, pos: VERB
word: giữa, lemma: giữa, pos: ADP
word: nhiều, lemma: nhiều, pos: ADJ
word: người, lemma: người, pos: NOUN
word: chơi, lemma: chơi, pos: VERB
word: đặc biệt, lemma: đặc biệt, pos: ADJ
word: giữa, lemma: giữa, pos: ADP
word: những, lemma: những, pos: DET
word: tuyển thủ, lemma: tuyển thủ, pos: NOUN
word: chuyên nghiệp, lemma: chuyên nghiệp, pos: PUNCT
word: Các thể, lemma: Các thể, pos: DET
word: loại, lemma: loại, pos: NOUN
word: trò chơi video, lemma: trò chơi video, pos: NOUN
word: phổ biến, lemma: phổ biến, pos: ADJ
word: nhất, lemma: nhấ

In [11]:
# Trích xuất và in ra kết quả các bộ ba từ câu
triples = []
sentence_triples = []

for sentence in doc.sentences:
    current_sentence_triples = []
    subjects = []
    objects = []
    verb = None
    for word in sentence.words:
        cleaned_word = clean_text(word.text)
        if word.deprel == 'nsubj':  # Chủ ngữ
            subjects.append(cleaned_word)
        elif word.pos == 'VERB':  # Động từ chính
            verb = cleaned_word
        #elif word.deprel in ['obj', 'dobj']
        elif word.deprel == 'obj' or word.deprel == 'dobj':  # Tân ngữ
            objects.append(cleaned_word)


    if subjects and verb and objects:
        for subj in subjects:
            for obj in objects:
                triple = (subj, verb, obj)
                triples.append(triple)
                current_sentence_triples.append(triple)

    sentence_triples.append(current_sentence_triples)

# Hàm in bộ ba quan hệ theo định dạng
def print_triples(triples, sentence_index):
    print(f"Sentence {sentence_index}:")
    for subj, verb, obj in triples:
        print(f"  ({subj}; {verb}; {obj})")

In [12]:
# Hàm vẽ các bộ ba đã trích xuất cho từng câu
def draw_dependency_tree(triples, sentence_index):
    # Khởi tạo đồ thị
    G = nx.DiGraph()

    # Thêm các cạnh (edges) vào đồ thị từ các bộ ba
    for subj, verb, obj in triples:
        G.add_edge(subj, verb)
        G.add_edge(verb, obj)

    # Vẽ đồ thị
    pos = nx.spring_layout(G)  # Sắp xếp vị trí các nút
    plt.figure(figsize=(10, 5))

    # Vẽ các nút và nhãn của chúng
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=3000, font_size=10, font_weight='bold', arrows=True)

    # Hiển thị tiêu đề và lưu ảnh
    plt.title(f"Cây quan hệ các thành phần{sentence_index}")
    plt.show
    plt.savefig(f"Cây quan hệ các thành phần_{sentence_index}.png")
    plt.close()



In [17]:
# In các bộ ba và vẽ sơ đồ quan hệ cho từng câu
for index, triples in enumerate(sentence_triples):
    print_triples(triples, index + 1)
    draw_dependency_tree(triples, index + 1)

print("Đã in các bộ ba quan hệ và vẽ lưu các sơ đồ cho từng câu.")

Sentence 1:
  (Thể; chơi; trận)
Sentence 2:
  (trò chơi video; thao điện; cuộc)
  (trò chơi video; thao điện; trò chơi chiến)
  (trò chơi video; thao điện; lược)
  (trò chơi video; thao điện; súng)
  (trò chơi video; thao điện; góc)
  (trò chơi video; thao điện; người)
  (trò chơi video; thao điện; đặc)
  (trò chơi video; thao điện; n)
  (trò chơi video; thao điện; ngôi)
  (trò chơi video; thao điện; nội)
  (trò chơi video; thao điện; tử như)
  (trò chơi video; thao điện; PUBG)
  (loại; thao điện; cuộc)
  (loại; thao điện; trò chơi chiến)
  (loại; thao điện; lược)
  (loại; thao điện; súng)
  (loại; thao điện; góc)
  (loại; thao điện; người)
  (loại; thao điện; đặc)
  (loại; thao điện; n)
  (loại; thao điện; ngôi)
  (loại; thao điện; nội)
  (loại; thao điện; tử như)
  (loại; thao điện; PUBG)
  (FPS; thao điện; cuộc)
  (FPS; thao điện; trò chơi chiến)
  (FPS; thao điện; lược)
  (FPS; thao điện; súng)
  (FPS; thao điện; góc)
  (FPS; thao điện; người)
  (FPS; thao điện; đặc)
  (FPS; thao đ

In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Giả sử bạn đã có ground truth dưới dạng danh sách các bộ ba (tuples)
# Ví dụ ground_truth = [(subj1, verb1, obj1), (subj2, verb2, obj2), ...]

# Hàm để chuyển đổi danh sách các bộ ba thành danh sách các tuples cho dễ so sánh
def triples_to_tuples(triples):
    return [(subj, verb, obj) for subj, verb, obj in triples]

# Ví dụ ground_truth (thay thế bằng dữ liệu thực tế)
ground_truth = [
    ("thể thao", "là", "trận đấu"),
    ("thể thao", "chơi", "trận "),
    ("thể thao", "là", "hình thức"),
    ("cuộc thi", "được", "diễn ra"),
    ("giải đấu", "có", "nguồn gốc"),

    # Thêm các bộ ba khác
]

# Chuyển đổi bộ ba trích xuất được và ground truth thành dạng tuples
extracted_triples = triples_to_tuples(triples)
ground_truth_tuples = triples_to_tuples(ground_truth)

# Tính toán Precision, Recall, F1-score
def evaluate_performance(extracted, ground_truth):
    extracted_set = set(extracted)
    ground_truth_set = set(ground_truth)

    true_positives = len(extracted_set & ground_truth_set)
    false_positives = len(extracted_set - ground_truth_set)
    false_negatives = len(ground_truth_set - extracted_set)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1, len(extracted)

precision, recall, f1, triple_count = evaluate_performance(extracted_triples, ground_truth_tuples)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Triple Count: {triple_count}")



Precision: 0.00
Recall: 0.00
F1-score: 0.00
Triple Count: 1
